In [1]:
import spotipy
import os
from spotipy.oauth2 import SpotifyOAuth
from IPython.core.display import display, HTML


In [2]:
def get_top_n(response):
    top_artists = []
    for item in response['items']:
        top_artists.append(item['name'])
    return top_artists

def parse_playlist_ids(response):
    playlist_ids = []
    for item in response['items']:
        pid = item['id']

        playlist_ids.append(pid)
    return playlist_ids

def parse_track_info(response):
    track_names = []
    artist_names = []
    album_names = []
    
    for item in r['items']:
                
        # Gets the name of the track
        track = item['track']['name']
        # Gets the name of the album
        album = item['track']['album']['name']
        # Gets the name of the first artist listed under album artists
        artist = item['track']['album']['artists'][0]['name']
             
        track_names.append(track)
        album_names.append(album)
        artist_names.append(artist) 
    return track_names, album_names, artist_names

In [3]:
# Application information
client_id = 'f78a4f4cfe9c40ea8fe346b0576e98ea'
client_secret = 'c26db2d4c1fb42d79dc99945b2360ab4'

# Temporary placeholder until we actually get a website going
redirect_uri = 'https://google.com/'

# The permissions that our application will ask for
scope = " ".join(['playlist-modify-public',"user-top-read","user-read-recently-played","playlist-read-private"])



# From Colin

Here I took some code from Colin's application. We can use this in the future when we actually have a website and we're sending people around various pages. Until then we will just use the SpotifyOAuth object to authenticate locally.

In [4]:
# Oauth object    
sp_oauth = spotipy.oauth2.SpotifyOAuth(client_id, client_secret, redirect_uri, scope=scope, cache_path=None, show_dialog=True)

# Force auth every time
authUrl = sp_oauth.get_authorize_url()
authUrl

'https://accounts.spotify.com/authorize?client_id=f78a4f4cfe9c40ea8fe346b0576e98ea&response_type=code&redirect_uri=https%3A%2F%2Fgoogle.com%2F&scope=playlist-modify-public+playlist-read-private+user-read-recently-played+user-top-read&show_dialog=True'

In [5]:
websiteHtml = """<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<title>Bad Vibes!</title>
<body>
<h1>Bad Vibes!</h1><br>
<a href="{}">Authorize</a><br><br>
You can trust me.
</body>""".format(authUrl)
site = HTML(websiteHtml)
display(site)

In [6]:
!pip show spotipy

Name: spotipy
Version: 2.16.1
Summary: A light weight Python library for the Spotify Web API
Home-page: https://spotipy.readthedocs.org/
Author: @plamere
Author-email: paul@echonest.com
License: LICENSE.md
Location: c:\users\dmcca\appdata\local\programs\python\python39\lib\site-packages
Requires: requests, six
Required-by: 


In [7]:
returnUrl = 'https://www.google.com/?code=AQCmTSTuoDjtNZhvq54_q7aPukaJxOc097FSlJ-LJFdLq6zvYnYC2lNcuKszNjRAlf_kfJmHnQhQ4zBfnSK6MF6EauJDEiKq3sWKZqMrt3q_9amxhKtPPyhlaT9HhLs16bGVJRhkqMHNpiAZBbQ_agq3wQYU9_xbSfJLt8h46by3Rk7o_cbkiucXZT0mxfIFdy4fJ6C0UdQttAfpcG9qOVwDhPxHy_DxyiXfqAPq1g1hon4ZYxeuw9WS6Kjkt_r4Lvw5TxkI15s_mYisXCmwgA'

In [8]:
code = returnUrl.split("?code=")[1].split("&")[0]
token_info = sp_oauth.get_access_token(code)
access_token = token_info['access_token']
sp = spotipy.Spotify(auth=access_token)

# Test auth
username = sp.current_user()['id']
username


<ipython-input-8-daba16ad0adc>:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(code)


'itzpersonal'

In [9]:
r = sp.search('playboy carti', type='artist')

In [10]:
(r['artists']['items'][0]['name'])

'Playboi Carti'

In [11]:
r['artists']['items'][0]['genres']

['atl hip hop', 'hip hop', 'rap', 'trap']

# Using Locally

For the time being this is how we will access user information on the Spotify API.

In [ ]:
sp = spotipy.Spotify(oauth_manager=sp_oauth)

# Methods of Pulling User Data

### Top Artists

In [ ]:
r = sp.current_user_top_artists(limit=50, time_range='short_term')
get_top_n(r)[:10]

In [ ]:
r = sp.current_user_top_artists(limit=50, time_range='medium_term')
get_top_n(r)[:10]

In [ ]:
r = sp.current_user_top_artists(limit=50, time_range='long_term')
get_top_n(r)[:10]

### Top Tracks

In [ ]:
r = sp.current_user_top_tracks(limit=50, time_range='short_term')
get_top_n(r)[:10]

In [ ]:
r = sp.current_user_top_tracks(limit=50, time_range='medium_term')
get_top_n(r)[:10]

In [ ]:
r = sp.current_user_top_tracks(limit=50, time_range='long_term')
get_top_n(r)[:10]

### Recently Played

In [ ]:
r = sp.current_user_recently_played(limit=50, after=None, before=None)


In [ ]:
tracks, albums, artists = parse_track_info(r)

In [ ]:
tracks[:10]

In [ ]:
albums[:10]

In [ ]:
artists[:10]

### Playlist Songs


Gettings the songs/artists/albums/etc. associated with a user's playlists can be accomplished in two steps.

1. Use ```sp.current_user_playlists()``` to find all the playlist IDs for the user's playlists.

2. Use ```sp.playlist_items()``` to find the songs on every playlist. Any other information could then be inferred from there.

In [ ]:
# First use this command to get all of the user's playlist IDs
r = sp.current_user_playlists()

In [ ]:
# Extract all playlist IDs in a list
playlist_ids = parse_playlist_ids(r)

In [ ]:
# Pull all the tracks from a playlist
tracks = []
albums = []
artists = []

# Loop through each playlist one by one
for pid in playlist_ids:
    # Request all track information
    r = sp.playlist_items(pid)
    
    tracks, albums, artists = parse_track_info(r)
    break
    
    

In [ ]:
tracks[:10]

In [ ]:
albums[:10]

In [ ]:
artists[:10]

In [ ]:
https://open.spotify.com/user/kynicoles?si=gJFaPiJ_T4GRkT8pQ_ZwbQ